In [1]:
import cv2
import numpy as np
path = 'data/lena.png'
img = cv2.imread(path)
#print(img)
timg =  img.astype('float32')
#print(timg)
img_hls = cv2.cvtColor(img,cv2.COLOR_BGR2HLS)
img_hsv = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)

In [2]:
#題目1:改變 color space 來調整飽和度
def convert_img(img_,last_value,saturation,type_):
    #操作飽和度時，一定要將整數轉換成小數
    img_ =  img_.astype('float32') #陣列本是整數值，改轉換成浮點數                
    img_[:,:,last_value] = img_[:,:,last_value]/255+saturation      
    #以下取特徵值，當特徵透過飽合度運算後，小於0時，全部設定為0    
    img_[img_[..., last_value] < 0] = 0   
    #將圖轉換回來，轉回0-255區間   
    img_[:,:,last_value] = img_[:,:,last_value]*255   
    #轉回unit8格式(0-255)
    img_ = img_.astype('uint8')
    if (type_ == "HSV"):
        img_ = cv2.cvtColor(img_,cv2.COLOR_HSV2BGR)
    elif (type_ =="HLS"):
        img_ = cv2.cvtColor(img_,cv2.COLOR_HLS2BGR)    
    elif (type_ =="FHLS"):
        img_ = cv2.cvtColor(img_,cv2.COLOR_HLS2BGR_FULL)    
    else:
        img_ = cv2.cvtColor(img_,cv2.COLOR_LAB2BGR)    
    return img_

In [3]:
#調降2成飽和度
img_cell_1_2 = convert_img(img_hsv, 1, -0.2, "HSV")#OK
img_cell_1_3 = convert_img(img_hls, -1,0.2, "HLS") #OK
merge_img1 = np.hstack((img,img_cell_1_2,img_cell_1_3)) #OK

In [4]:
img_cell_2_2 = convert_img(img_hls, -1,-0.2, "HLS")#OK
img_hsv2 = img_hsv.copy()
img_hsv2 =cv2.convertScaleAbs(img_hsv2, alpha=1.0, beta=3)
img_cell_2_3 = convert_img(img_hsv2,-1,0.03, "HSV")#有點像
merge_img2 = np.hstack((img,img_cell_2_2,img_cell_2_3))

In [5]:
channel_list = [img[...,0],img[...,1],img[...,2]] #取g,b,r不同channel至陣列各陣列
#ex:[1,2,3],[4,5,6]-->[ [1,4],[2,5],[3,6]]
channel_list[0] = cv2.equalizeHist(img[...,0])
channel_list[1] = cv2.equalizeHist(img[...,1])
channel_list[2] = cv2.equalizeHist(img[...,2])
#channel_list = (cv2.equalizeHist(img[...,0]), cv2.equalizeHist(img[:, :, 1]), cv2.equalizeHist(img[:, :, 2]))

#channel_list = [cv2.equalizeHist(i) for i in channel_list] #看不懂此行
img_cell_3_2=np.stack( channel_list, axis=-1)  # axis=-1 = axis=1,將3個channel的垂直陣列再合併起來
#cv2.imshow('color space',img_cell_3_2);#OK
#cv2.waitKey(0)
#print(img_cell_3_2)
#ex:[ [1,4],[2,5],[3,6]]-->[1,2,3][4,5,6]
#https://blog.csdn.net/qq1483661204/article/details/78149262

In [6]:
img_hsv2 = img_hsv.copy()
img_hsv2[...,-1] =cv2.equalizeHist(img_hsv2[...,-1])
img_cell_3_3 = cv2.cvtColor(img_hsv2,cv2.COLOR_HSV2BGR)#OK
#print(img_cell_3_3)
merge_img3 = np.hstack((img,img_cell_3_2,img_cell_3_3)) #推壘起來

In [7]:
#題目2:調整明亮度
img_cell_4_2=cv2.convertScaleAbs(img, alpha=1.5, beta=30)#OK
img_cell_4_3=cv2.convertScaleAbs(img, alpha=-1, beta=-40)#OK
merge_img4 = np.hstack((img,img_cell_4_2,img_cell_4_3))

In [8]:
merge = np.vstack((merge_img1,merge_img2,merge_img3,merge_img4))
dim=(500,600)
resized = cv2.resize(merge, dim, interpolation = cv2.INTER_AREA)
cv2.imshow('color space',resized);
cv2.waitKey(0)

-1

In [9]:
#img_cell_3_2 = convert_img(img_hsv,"b",1,0.2, "HSV") 
#img_gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
def convert_img2(img_,last_value):
    img_ =  img_.astype('float32') #陣列本是整數值，改轉換成浮點數            
    #img_[...,last_value] = img_[...,last_value]/255+saturation #代入1時，會有些相素沒算到，轉換圖片時會有黑影       
    img_[last_value] = img_[last_value]/255+saturation
        
    #以下取特徵值，當特徵透過飽合度運算後，小於0時，全部設定為0    
    img_[img_[last_value] < 0] = 0
    #img_[img_[last_value,:,:] < 0] = 0
    #將圖轉換回來，轉回0-255區間   
    img_[last_value] = img_[last_value]*255
    
    #轉回unit8格式(0-255)
    img_ = img_.astype('uint8')
    img_ = cv2.cvtColor(img_,cv2.COLOR_GRAY2BGR)    
    return img_

img_gray= cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
img_equal = cv2.equalizeHist(img_gray)
img2=cv2.cvtColor(img_equal,cv2.COLOR_GRAY2BGR)
#merge_g = np.hstack((img_gray,img_equal,img2)) 
#cv2.imshow('color space',img2);
#cv2.waitKey(0)

In [10]:
#調整明亮度另一個作法就是用bgr換算 = convertScaleAbs()函式
#下列作法執行上會較慢
#灰階(剩2維)不用下列方式，因為只剩黑、白色系

def convertScale(img_,alpha,beta):    
    for b in range(img_.shape[0]):#blue
        for g in range(img_.shape[1]):#green
            for r in range(img_.shape[2]):#red
                img_[b,g,r] = np.clip(alpha * img_[b,g,r] + beta, 0 ,255) 
                #clip():經過換算後，所有的值一定要介於0-255之間，>0者，=0；>255者，=255
    return img_

In [11]:
#fimg2 = img.copy() #https://www.programiz.com/python-programming/methods/list/copy
#fimg2 = convertScale(fimg2,2.0,0)
#fimg3 = img.copy()
#fimg3 = convertScale(fimg3,1.0,50)
#merge_imgf = np.hstack((img,fimg2,fimg3))
#cv2.imshow('明亮度[bgr]-old',merge_imgf)
#cv2.waitKey(0) 

In [12]:
#題目3:直方圖均衡
#轉換成灰階
#問題：為何一定要轉換成灰階？
img_gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
#print(img_gray) #img原圖是3維，灰階後剩2維
img_gray_equal = cv2.equalizeHist(img_gray)
print(img_gray_equal)
merge_img = np.hstack((img_gray,img_gray_equal))  #各為2維陣列，故無法與img進行hstack
#cv2.imshow('直方圖均衡',merge_img)
#cv2.waitKey(0)

[[200 200 204 ... 209 177 123]
 [200 200 204 ... 209 177 123]
 [200 200 204 ... 209 177 123]
 ...
 [  9   9  22 ...  87  76  72]
 [ 11  11  30 ...  82  85  89]
 [ 11  11  30 ...  82  85  89]]
